In [3]:
import numpy as np
import pandas as pd

In [4]:
from fossa.utils import dummy_data

In [5]:
MIN_VAL = 1000
MAX_VAL = 1200
NUM_CATEG = 3

In [6]:
sorted_df = dummy_data(60, NUM_CATEG, MIN_VAL, MAX_VAL)
# mock.sample(5)

In [7]:
sorted_df.index[0][0]

Timestamp('2011-01-01 00:00:00')

In [8]:
new_data = dummy_data(4, NUM_CATEG, MIN_VAL, MAX_VAL)

In [9]:
new_window = new_data.loc[new_data.index.levels[0][0]]
new_window

,value
category,
0,1196
1,1073
2,1055


In [10]:
from scipy.stats import power_divergence

In [11]:
p_threhold = 0.001

In [12]:
def _detect_trend(obs, exp):
    direction = 1
    if obs[0] < exp[0]:
        direction = -1
    res = power_divergence(obs, exp)
    if res[1] < p_threhold:
        return res[1], direction
    return res[1], 0

In [13]:
committe = [
    (1, sorted_df.loc[sorted_df.index.levels[0][i]])
    for i in range(15)
]

In [14]:
weigh_by_p_val = False

In [15]:
from fossa.utils import pad_windows, one_vs_all_dists
from strct.dicts import increment_nested_val

In [16]:
votes_by_category = {}
for weight, committee_win in committe:
    new_padded, member_padded = pad_windows(new_window, committee_win)
    new_1vall = one_vs_all_dists(new_padded)
    member_1vall = one_vs_all_dists(member_padded)
    for categ in new_1vall:
        p_val, trend = _detect_trend(
            obs=new_1vall[categ],
            exp=member_1vall[categ],
        )
        vote = weight
        if weigh_by_p_val:
            if trend == 0:
                vote *= p_val
            else:
                vote *= (1 - p_val)
        increment_nested_val(
            dict_obj=votes_by_category,
            key_tuple=(categ, trend),
            value=vote,
        )

In [17]:
votes_by_category

{'0': {-1: 1, 0: 5, 1: 9}, '1': {-1: 2, 0: 12, 1: 1}, '2': {-1: 6, 0: 7, 1: 2}}

In [18]:
from strct.dicts import get_key_val_of_max

In [22]:
trends

{'0': (0.6, 1), '1': (0.8, 0), '2': (0.4666666666666667, 0)}

In [23]:
trends_df

,conf,direction
0,0.600000,1.0
1,0.800000,0.0
2,0.466667,0.0


In [41]:
new_data

value
date       category       
2011-01-01 0          1196
           1          1073
           2          1055
2011-01-02 0          1099
           1          1167
           2          1058
2011-01-03 0          1022
           1          1199
           2          1163
2011-01-04 0          1091
           1          1128
           2          1132

In [42]:
for ix in new_data.index.levels[0]:
    print(new_data.loc[ix])

          value
category       
0          1196
1          1073
2          1055
          value
category       
0          1099
1          1167
2          1058
          value
category       
0          1022
1          1199
2          1163
          value
category       
0          1091
1          1128
2          1132


In [79]:
subdf = new_data.loc[new_data.index.levels[0][0]]
subdf

,value
category,
0,1196
1,1073
2,1055


In [81]:
subdf.index

Index(['0', '1', '2'], dtype='object', name='category')

In [85]:
new_data.index.levels[0][-2:]

DatetimeIndex(['2011-01-03', '2011-01-04'], dtype='datetime64[ns]', name='date', freq=None)

In [91]:
new_data

value
date       category       
2011-01-01 0          1196
           1          1073
           2          1055
2011-01-02 0          1099
           1          1167
           2          1058
2011-01-03 0          1022
           1          1199
           2          1163
2011-01-04 0          1091
           1          1128
           2          1132

In [104]:
new_data.index.levels[0][-2:]

DatetimeIndex(['2011-01-03', '2011-01-04'], dtype='datetime64[ns]', name='date', freq=None)

In [110]:
new_data.loc?

Type:        property
String form: <property object at 0x109c24ef8>
Docstring:  
Purely label-location based indexer for selection by label.

``.loc[]`` is primarily label based, but may also be used with a
boolean array.

Allowed inputs are:

- A single label, e.g. ``5`` or ``'a'``, (note that ``5`` is
  interpreted as a *label* of the index, and **never** as an
  integer position along the index).
- A list or array of labels, e.g. ``['a', 'b', 'c']``.
- A slice object with labels, e.g. ``'a':'f'`` (note that contrary
  to usual python slices, **both** the start and the stop are included!).
- A boolean array.
- A ``callable`` function with one argument (the calling Series, DataFrame
  or Panel) and that returns valid output for indexing (one of the above)

``.loc`` will raise a ``KeyError`` when the items are not found.

See more at :ref:`Selection by Label <indexing.label>`


In [113]:
new_data.loc[pd.IndexSlice[new_data.index.levels[0][-2:],:], :]

value
date       category       
2011-01-03 0          1022
           1          1199
           2          1163
2011-01-04 0          1091
           1          1128
           2          1132